In [1]:
import attiicc as ac
from attiicc.segmentation.sam import SamSegmenter
import os

In [2]:
image_path = "/media/ss4tbSSD/EDYER/nanowell_processing/images/p00_0_A01f00d0_2.png"
segment_img_save = "/media/ss4tbSSD/EDYER/nanowell_processing/sam_test_seg.png"
masks_img_save = "/media/ss4tbSSD/EDYER/nanowell_processing/sam_test_masks.png"
roi_path = "/home/ecdyer/labshare/LL07132018/NanoTry20231211/20X_H2O2_3_hours/ROIS"
model_path = "/media/ss4tbSSD/EDYER/nanowell_processing/weights/sam_vit_h_4b8939.pth"

In [3]:
test_sam = SamSegmenter(model_path=model_path, image_path=image_path)

CUDA is available.
Number of CUDA devices: 1
Model Loaded


In [ ]:
test_sam.plot_segmented_image(save=True, save_path=segment_img_save)
test_sam.plot_masks(save=True, save_path=masks_img_save)

In [4]:
test_dict = test_sam.generate_rois(roi_path=roi_path)
print(test_dict)

Duplicate Value 1: (205, 905) Duplicate Value 2: (205, 905)
Duplicate Value 1: (493, 427) Duplicate Value 2: (494, 427)
Duplicate Value 1: (570, 65) Duplicate Value 2: (571, 66)
Duplicate Value 1: (850, 69) Duplicate Value 2: (851, 69)
Total number of ROIs:  71
Saving ROIs to:  /home/ecdyer/labshare/LL07132018/NanoTry20231211/20X_H2O2_3_hours/ROIS/p00_0_A01f00d0_2
ROIs saved for p00_0_A01f00d0_2
Archiving ROIs to:  /home/ecdyer/labshare/LL07132018/NanoTry20231211/20X_H2O2_3_hours/ROIS/p00_0_A01f00d0_2_roi.zip
ROIs archived for p00_0_A01f00d0_2
{0: [432, 63, ImagejRoi(
    roitype=ROI_TYPE.FREEHAND,
    options=ROI_OPTIONS.OVERLAY_BACKGROUNDS | ROI_OPTIONS.OVERLAY_LABELS,
    name='fef9c478-acbe-11ee-be85-490a135c3209',
    version=226,
    top=3,
    left=372,
    bottom=129,
    right=495,
    n_coordinates=178,
    integer_coordinates=numpy.array([
        [ 46,   0],
        [ 45,   1],
        [ 42,   1],
        [ 41,   2],
        [ 39,   2],
        [ 38,   3],
        [ 37,   3